# TRANSFER LEARNING FOR IMPROVED MODEL WITH LESS DATA

_**Applies transfer learning to reuse pretrained layers to experiment if it improves model performance with less data.**_

**High-level Steps:**

1. Instead of taking an already trained model (containing pretrained layers), a model gets trained in this experiment to be considered as a pretrained model. To train that model, data for 8 classes out of total 10 classes in Fashion MNIST dataset are used. This is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images.
a drop-in replacement for MNIST.

2. Then a binary classification model (the target model) gets trained (from scratch) on the data from remaining two clssses from the same dataset and its prediction performance gets observed.

3. Then the same classification model is build by apply transfer learning using pretrained layers from the model created in first step.

4. Lastly the prediction performance of the target model is compared with that of the model created in the second step. Also, analysis is performed to appreciate if transfer learning speeds up training and make training possible with less data.

In [ ]:
# Imports required packages

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2025-01-29 11:31:30.670478: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 11:31:30.676403: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 11:31:30.756806: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 11:31:30.854145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738130490.958316  763483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738130490.98

## Loading and Preparing Data

In [2]:
# Loads fashion mnist dataset
fashion = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
# Each training and test example is assigned to one of the following labels.
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", \
               "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [4]:
# Considering dataset is organized in tuple, items are referenced as follows
(X_train_full, y_train_full), (X_test, y_test) = fashion

In [5]:
# Checks the shape of the datasets
print("Train dataset shape:", X_train_full.shape)
print("Test dataset shape:", X_test.shape)

Train dataset shape: (60000, 28, 28)
Test dataset shape: (10000, 28, 28)


In [6]:
# Checks the data type of the data
X_train_full.dtype

dtype('uint8')

In [7]:
# Considering the data type of the data, it normalizes the data between 0 and 1 
# to make neural network model training efficient
X_train_full, X_test = X_train_full / 255., X_test / 255.

In [8]:
# Prints the labels for refer to the class index
y_train_full

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

Considering the target binary classification model is expected to classify "Pullover" and "T-shirt/top", it separates data for these two classes leaving data for remaining 8 classes to build a model to be considered as pretrained model later.

In [9]:
# Finds the index for the target class "Pullover" and "T-shirt/top" as
# dataset labels contains class indexes instead of class names

class_0_index = class_names.index("Pullover")
class_1_index = class_names.index("T-shirt/top")

print("Index of class_0:", class_0_index)
print("Index of class_1:", class_1_index)

Index of class_0: 2
Index of class_1: 0


In [10]:
# Gets the indexes of training label containing either classes
class_0_1_index_flag = [True if (x==class_0_index or x==class_1_index) else False for x in y_train_full]

# Shows few flags 
print(class_0_1_index_flag[:10])

[False, True, True, False, True, True, False, True, False, False]


In [11]:
# Seperates dataset containing data for two classes
X_train_2_classes_full = X_train_full[class_0_1_index_flag]

# Checks the shape of the dataset
X_train_2_classes_full.shape

(12000, 28, 28)

In [12]:
# Flips bool values (True to False and False to True) to get the flags against
# other classes in the training label
class_0_1_index_flag_flipped = [not flag for flag in class_0_1_index_flag]

# Shows few flags 
print(class_0_1_index_flag_flipped[:10])

[True, False, False, True, False, False, True, False, True, True]


In [13]:
# Seperates dataset containing data for the remaining 8 classes
X_train_8_classes_full = X_train_full[class_0_1_index_flag_flipped]

# Checks the shape of the dataset
X_train_8_classes_full.shape

(48000, 28, 28)

In [14]:
# Sum of the first dimension value of both the dataset should be equal to the total number of training instances
X_train_2_classes_full.shape[0] + X_train_8_classes_full.shape[0]

60000

In [15]:
# Similarly, separates targets to contain only respective labels
y_train_2_classes_full = y_train_full[class_0_1_index_flag]
y_train_8_classes_full = y_train_full[class_0_1_index_flag_flipped]

# Checks the shape of the targets
print(y_train_2_classes_full.shape)
print(y_train_8_classes_full.shape)

(12000,)
(48000,)


## Modeling

### Training Model to be Considered as Pretrained

**Preprocesses Datasets**

In [16]:
# Separates validation dataset
X_train_8_classes, X_val_8_classes, y_train_8_classes, y_val_8_classes = train_test_split(
    X_train_8_classes_full, y_train_8_classes_full, test_size=5000, random_state=42, stratify=y_train_8_classes_full)

In [17]:
# Prints the shape of the separated datasets both containing 8 classes
print(X_train_8_classes.shape)
print(X_val_8_classes.shape)

(43000, 28, 28)
(5000, 28, 28)


In [48]:
# Then standardizes the datasets by first calculating mean and standard deviation, and then
# by subtracting the mean from the data and then dividing the data by standard deviation

pixel_means_8_classes = X_train_8_classes.mean(axis=0, keepdims=True)
pixel_stds_8_classes = X_train_8_classes.std(axis=0, keepdims=True)

X_train_8_classes_scaled = (X_train_8_classes - pixel_means_8_classes) / pixel_stds_8_classes
X_val_8_classes_scaled = (X_val_8_classes - pixel_means_8_classes) / pixel_stds_8_classes


In [49]:
# As the labels ranges from [1, 3, 4, 5, 6, 7, 8, 9], it normalizes the label from 0 through 7
label_encoder_8_classes = LabelEncoder()
y_train_8_classes_encoded = label_encoder_8_classes.fit_transform(y_train_8_classes)
y_val_8_classes_encoded = label_encoder_8_classes.transform(y_val_8_classes)

In [50]:
# Initializes the following densed neural network with arbirary number of layers and compiles it

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics=["accuracy"])


/home/pradip/anaconda3/envs/keras3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
# Checks for model summary [optional]
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           808 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,508 (388.70 KB)

 Trainable params: 99,508 (388.70 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Fits the model over specific number iterations (epochs) and validation data
# to observe the learning performance during training
model_history = model.fit(X_train_8_classes_scaled, y_train_8_classes_encoded, epochs=20, 
                          validation_data=(X_val_8_classes_scaled, y_val_8_classes_encoded))

Epoch 1/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6041 - loss: 1.1918 - val_accuracy: 0.8284 - val_loss: 0.4953
Epoch 2/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8395 - loss: 0.4699 - val_accuracy: 0.8580 - val_loss: 0.3915
Epoch 3/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8616 - loss: 0.3857 - val_accuracy: 0.8764 - val_loss: 0.3438
Epoch 4/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8818 - loss: 0.3327 - val_accuracy: 0.8880 - val_loss: 0.3162
Epoch 5/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8903 - loss: 0.3069 - val_accuracy: 0.8902 - val_loss: 0.2997
Epoch 6/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8969 - loss: 0.2845 - val_accuracy: 0.8976 - val_loss: 0.2877
Epoch 7/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9032 - loss: 0.2733 - val_accuracy: 0.9000 - val_loss: 0.2768
Epoch 8/20
1344/1344 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9081 - loss: 0.2634 - 

In [53]:
# Saves the trained model on disk to be used as pretrained model later.
# NOTE: Folder "model" must exist for model file to be saved into.
model.save("./models/my_fashion_mnist_model.keras")

### Training Target Model from Scratch

**Preprocesses Datasets**

In [54]:
# Separates validation dataset from the data containg 2 classes
X_train_2_classes, X_val_2_classes, y_train_2_classes, y_val_2_classes = train_test_split(
    X_train_2_classes_full, y_train_2_classes_full, test_size=3000, random_state=42, stratify=y_train_2_classes_full)

In [55]:
# Prints the shape of the separated datasets containing both classes
print(X_train_2_classes.shape)
print(X_val_2_classes.shape)

(9000, 28, 28)
(3000, 28, 28)


In [57]:
# Then standardizes the datasets by first calculating mean and standard deviation, and then
# by subtracting the mean from the data and then dividing the data by standard deviation

pixel_means_2_classes = X_train_2_classes.mean(axis=0, keepdims=True)
pixel_stds_2_classes = X_train_2_classes.std(axis=0, keepdims=True)

X_train_2_classes_scaled = (X_train_2_classes - pixel_means_2_classes) / pixel_stds_2_classes
X_val_2_classes_scaled = (X_val_2_classes - pixel_means_2_classes) / pixel_stds_2_classes

In [60]:
# As the labels ranges from [1, 3, 4, 5, 6, 7, 8, 9], it normalizes the label from 0 through 7

label_encoder_2_classes = LabelEncoder()
y_train_2_classes_encoded = label_encoder_2_classes.fit_transform(y_train_2_classes)
y_val_2_classes_encoded = label_encoder_2_classes.transform(y_val_2_classes)

In [61]:
# Clears the name counters and 
# sets the global random seed for operations that rely on a random seed
tf.keras.backend.clear_session()
tf.random.set_seed(42)

# Initializes the following densed neural network with arbirary number of layers and compiles it
model_from_scratch = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_from_scratch.compile(
    loss="binary_crossentropy", 
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), 
    metrics=["accuracy"])

/home/pradip/anaconda3/envs/keras3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [62]:
# Checks for model summary [optional]
model_from_scratch.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,801 (385.94 KB)

 Trainable params: 98,801 (385.94 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
# Fits the model over specific number iterations (epochs) on all the training data available for the 2 classes
# and validation data to observe the learning performance during training
model_from_scratch_history = model_from_scratch.fit(X_train_2_classes_scaled, y_train_2_classes_encoded, epochs=20, 
                                                    validation_data=(X_val_2_classes_scaled, y_val_2_classes_encoded))

Epoch 1/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7772 - loss: 0.5186 - val_accuracy: 0.9450 - val_loss: 0.2422
Epoch 2/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9416 - loss: 0.2235 - val_accuracy: 0.9553 - val_loss: 0.1628
Epoch 3/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9505 - loss: 0.1605 - val_accuracy: 0.9567 - val_loss: 0.1371
Epoch 4/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9552 - loss: 0.1364 - val_accuracy: 0.9593 - val_loss: 0.1257
Epoch 5/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9579 - loss: 0.1236 - val_accuracy: 0.9610 - val_loss: 0.1196
Epoch 6/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9595 - loss: 0.1155 - val_accuracy: 0.9617 - val_loss: 0.1158
Epoch 7/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9612 - loss: 0.1097 - val_accuracy: 0.9627 - val_loss: 0.1130
Epoch 8/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9620 - loss: 0.1053 - val_accuracy: 0.

In [65]:
# Gets the indexes of test label containing either classes
class_0_1_index_flag = [True if (x==class_0_index or x==class_1_index) else False for x in y_test]

In [66]:
# Seperates dataset containing data for two classes from the whole test set also containing other classes
X_test_2_classes = X_test[class_0_1_index_flag]

# Checks the shape of the dataset
X_test_2_classes.shape

(2000, 28, 28)

In [67]:
# Similarly, separates targets to contain only respective labels
y_test_2_classes = y_test[class_0_1_index_flag]

In [68]:
# Normalizes the test labels for the 2 classes using the already fitted encoder
y_test_2_classes_encoded = label_encoder_2_classes.transform(y_test_2_classes)

In [69]:
# Prints the encoded classes for reference
y_test_2_classes_encoded

array([1, 1, 0, ..., 0, 0, 1])

In [70]:
# Standardizes the test set by subtracting the mean from the data and then dividing the data by standard deviation
X_test_2_classes_scaled = (X_test_2_classes - pixel_means_2_classes) / pixel_stds_2_classes

In [71]:
# Evaluates the test prediction performance on the model built from scratch
model_from_scratch.evaluate(X_test_2_classes_scaled, y_test_2_classes_encoded)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.9640 - loss: 0.1104


[0.1123715341091156, 0.9620000123977661]

The above model that was built from scratch over **9000** [12000 total - 3000 validation instances] training instances containing data for 2 classes, reached **96.20%** accuracy in test set. The experiment continues to apply transfer learning by reusing pretrained layers from first model built over other 8 classes to check if new model trained over less data can achieve accuracy from the model built from scratch.

### Transfer Learning

In [72]:
# Loads the saved model created to be used as pretrained model
model_using_pretrained_layers = tf.keras.models.load_model("./models/my_fashion_mnist_model.keras")

In [73]:
# Checks the model summary especially to refer to the last layer i.e. the output layer
model_using_pretrained_layers.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           808 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,510 (388.71 KB)

 Trainable params: 99,508 (388.70 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [75]:
# Removes the last layer (containing 8 output) to add task specific binary output layer
model_using_pretrained_layers.pop()

# And then adds a binary output layer
model_using_pretrained_layers.add(tf.keras.layers.Dense(1, activation="sigmoid", name="output"))

# Then verifies the same visualizing the model summary
model_using_pretrained_layers.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,703 (346.50 KB)

 Trainable params: 88,701 (346.49 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

**Fine-tuning already pretrained model**

In [76]:
# Considers only 60% of the 2-classes training set to check the effectiveness of the transfer learning

X_train_2_classes_scaled_subset, _, y_train_2_classes_encoded_subset, _ = train_test_split(
    X_train_2_classes_scaled, y_train_2_classes_encoded, train_size=0.60, stratify=y_train_2_classes_encoded)

In [77]:
# First sets all the pretrained layers (except for the newly added output layer) non-trainable
for layer in model_using_pretrained_layers.layers[:-1]:
    layer.trainable = False

In [78]:
# Then trains the just the output layer

tf.keras.backend.clear_session()
tf.random.set_seed(42)

model_using_pretrained_layers.compile(
    loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))

model_using_pretrained_layers_history = model_using_pretrained_layers.fit(
    X_train_2_classes_scaled_subset, y_train_2_classes_encoded_subset, epochs=5, 
    validation_data=(X_val_2_classes_scaled, y_val_2_classes_encoded))


Epoch 1/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3820 - loss: 1.1153 - val_accuracy: 0.6243 - val_loss: 0.6892
Epoch 2/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6997 - loss: 0.6376 - val_accuracy: 0.7933 - val_loss: 0.4926
Epoch 3/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8305 - loss: 0.4782 - val_accuracy: 0.8613 - val_loss: 0.3981
Epoch 4/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8729 - loss: 0.3963 - val_accuracy: 0.8857 - val_loss: 0.3436
Epoch 5/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8966 - loss: 0.3435 - val_accuracy: 0.8997 - val_loss: 0.3082


In [ ]:
# Now, makes all the pretrained layers trainable and performs retraining over small smaller 
# learning rate for longer iterations

for layer in model_using_pretrained_layers.layers[:-1]:
    layer.trainable = True

# Recompiles the model due to change of trainability of the layers
model_using_pretrained_layers.compile(
    loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))

model_using_pretrained_layers_history = model_using_pretrained_layers.fit(
   X_train_2_classes_scaled_subset, y_train_2_classes_encoded_subset, epochs=100, 
   validation_data=(X_val_2_classes_scaled, y_val_2_classes_encoded))

Epoch 1/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9184 - loss: 0.2853 - val_accuracy: 0.9407 - val_loss: 0.2119
Epoch 2/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9438 - loss: 0.2051 - val_accuracy: 0.9507 - val_loss: 0.1738
Epoch 3/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9519 - loss: 0.1684 - val_accuracy: 0.9543 - val_loss: 0.1546
Epoch 4/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9551 - loss: 0.1479 - val_accuracy: 0.9553 - val_loss: 0.1433
Epoch 5/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9584 - loss: 0.1352 - val_accuracy: 0.9570 - val_loss: 0.1361
Epoch 6/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9610 - loss: 0.1265 - val_accuracy: 0.9577 - val_loss: 0.1310
Epoch 7/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9644 - loss: 0.1202 - val_accuracy: 0.9590 - val_loss: 0.1273
Epoch 8/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9659 - loss: 0.1152 - val_accu

In [80]:
# Evaluates the test prediction performance on the model built using pretrained layers
model_using_pretrained_layers.evaluate(X_test_2_classes_scaled, y_test_2_classes_encoded)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - accuracy: 0.9687 - loss: 0.0929


[0.09692149609327316, 0.9674999713897705]

Though this model built over pretrained layers using on 60% of the available training set, but could also achieved **96.75%** test accuracy as compared to **96.2%** accuracy of the model built from scratch over the full training set. The error rate was improved by **14%** [(96.75−96.2)÷(100−96.20)×100].